In [ ]:
# Импорт необходимых библиотек
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Загрузка и предобработка данных
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Нормализация данных
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot кодирование меток
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Использование предобученной модели VGG16
# Включаем только свёрточные слои, без полносвязного блока (include_top=False)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Добавление новых слоев поверх базовой модели
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

# Создание модели
model = Model(inputs=base_model.input, outputs=predictions)

# Заморозка слоев базовой модели VGG16, чтобы они не обучались заново
for layer in base_model.layers:
    layer.trainable = False

# Компиляция модели
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Аугментация данных для улучшения обучения
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Обучение модели
batch_size = 64
epochs = 20

# Используем fit_generator для работы с аугментацией данных
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    validation_data=(x_test, y_test),
    epochs=epochs
)

# Оценка модели на тестовых данных
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_accuracy:.4f}')

# Сохранение модели
model.save('vgg16_cifar10.keras')


Epoch 1/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 705s 901ms/step - accuracy: 0.2424 - loss: 2.0902 - val_accuracy: 0.4750 - val_loss: 1.5478
Epoch 2/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 108s 138ms/step - accuracy: 0.4375 - loss: 1.5714 - val_accuracy: 0.4751 - val_loss: 1.5470
Epoch 3/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 710s 845ms/step - accuracy: 0.4401 - loss: 1.6127 - val_accuracy: 0.5113 - val_loss: 1.4209
Epoch 4/20
  1/781 ━━━━━━━━━━━━━━━━━━━━ 7:42 593ms/step - accuracy: 0.5000 - loss: 1.6043

In [3]:
for layer in model.layers:
    layer.trainable = True

# Переобучение модели с низким learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history_fine = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    validation_data=(x_test, y_test),
    epochs=10  # Можно продолжить обучение на нескольких эпохах
)

NameError: name 'model' is not defined

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2
)

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)


In [ ]:
from tensorflow.keras.regularizers import l2

x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)


In [ ]:
# Разморозка слоев начиная с определенного уровня (например, последних 4 блоков)
for layer in model.layers[:15]:
    layer.trainable = False
for layer in model.layers[15:]:
    layer.trainable = True

# Компиляция модели с меньшим learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Переобучение модели
history_fine = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    validation_data=(x_test, y_test),
    epochs=10  # Можно увеличить количество эпох
)
